# Assignment 2 (15 Marks)

In this assignment, you will implement Genetic Algorithm and simulate a simple PID controller. <br>
<br>

**The assignment is divided as follows:**<br>
Section 1 - 6. Implement Genetic Algorithm (9 Marks)<br>
Section 7 - 9. PID Control (5 Marks)<br>
Section 10. Using GA to tune PID (1 Mark)


Have fun experimenting!

#### Queries:
Queries regarding GA:
Dhaivata Pandya (f2016020@pilani.bits-pilani.ac.in) (WhatsApp: 8451949590)

Queries regarding PID:
Saksham Consul (f2016424@pilani.bits-pilani.ac.in) (WhatsApp: 7073998320)

# Basic GA Implementation

### 1- Import Packages:
**DO NOT** import any package from your side.

In [1]:
# do not change code here
import random
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

### 2- GAString class definition (1.75 Mark)

Each GAString instance represents a single chromosome.
- _n_ - represents the number of genes in the chromosome
- _genes_ - represents the bitstring of the chromosome as an np.array of booleans

The GAString class also has implementations for some magic functions. These are implemented to allow you to use functions like _print, sort_ and _min/max_ directly on GAString objects. They will make testing and debugging easier.

Note that if the fitness for two individuals is the same, you can use the <>= operators on the GAString objects directly to evaluate further, no need to scratch your head on that.

There is also a `randInit` function, which is used for initialising the genes if they are not provided directly.

For creating a deep copy of an instance, you will need to provide both `n` and the `genes` numpy.array object.

In [2]:
# do not change code here
class GAString():
    def __init__(self, number_of_genes, genes=None):
        self.n = number_of_genes
        self.genes = np.copy(genes) if genes is not None else self.randInit()
    
    def __str__(self):
        return ''.join(list(map(lambda x: '1' if x else '0', self.genes)))
    
    def __repr__(self):
        return f"GAString({self.n}, genes={self.__str__()})"
    
    def __lt__(self, other):
        for b1,b2 in zip(self.genes,other.genes):
            if b1 < b2:
                return True
            elif b1 > b2:
                return False
        return False
    
    def __le__(self, other):
        return self.__lt__(other) or self.__eq__(other)
    
    def __ge__(self, other):
        return self.__gt__(other) or self.__eq__(other)
    
    def __gt__(self, other):
        for b1,b2 in zip(self.genes,other.genes):
            if b1 > b2:
                return True
            elif b1 < b2:
                return False
        return False
    
    def __eq__(self, other):
        return self.n == other.n and np.array_equal(self.genes, other.genes)
    
    def __ne__(self, other):
        return self.n != other.n or not np.array_equal(self.genes, other.genes)
    
    def randInit(self):
        genes = np.empty(self.n, dtype=bool)
        for i in range(self.n):
            genes[i] = True if random.random() < 0.5 else False
        return genes

#### 2.1 Initialising the population (0.5 Mark)

**The tasks to be performed:**<br>
- Initialise the population (0.5 Mark)

In [3]:
### 0.5 Mark
def initPop(size_pop, number_of_genes):
    """
    Input:  size_pop is the size of the desired population i.e. number of chromosomes
            number_of_genes is the number of genes in each chromosome
    
    Output: a list of the required number of GAStrings, representing the population
    
    Note: You do not need to make any random.random() calls explicitly
          Use the class definition and functions to create the population
    """
    assert size_pop > 0 and number_of_genes > 0
    
    ### BEGIN SOLUTION
    return [GAString(number_of_genes) for _ in range(size_pop)]
    ### END SOLUTION

In [4]:
# do not change code here
### SAMPLE TEST CASE
random.seed(2)
print("Running sample test case")
res = initPop(6, 6)
ans = [GAString(6, genes=np.array([0,0,1,1,0,0],dtype=bool)), GAString(6, genes=np.array([0,1,0,0,0,1],dtype=bool)), 
       GAString(6, genes=np.array([1,1,0,0,0,0],dtype=bool)), GAString(6, genes=np.array([1,1,1,1,1,1],dtype=bool)), 
       GAString(6, genes=np.array([1,0,0,0,1,1],dtype=bool)), GAString(6, genes=np.array([1,1,0,0,0,1],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [5]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo(size_pop, number_of_genes):
    return [GAString(number_of_genes) for _ in range(size_pop)]

for _ in range(5):
    state = random.getstate()
    res = initPop(20,32)
    random.setstate(state)
    ans = _funkyfoo(20,32)
    assert all(a == b for a,b in zip(ans,res))

### END HIDDEN TESTS

#### 2.2 Decoding the bitstring (1.25 Mark)

So far, we have not specified what the binary coded string represents. Well, that's the thing, it can represent anything we want it to. What it means in real world terms depends on the problem being solved. The bitstring can represent the closest guess to an unknown number, or it can be the values of parameters for solving another optimization task.

**The tasks to be performed:**<br>

- Implement the _decodeInteger_ function, which will interpret the bitstring as the binary representation of an integer and return the value (0.25 Mark)
<br><br>
- Implement the _decodeFloatRange_ function, which converts the bitstring to a floating point value, with the min and max being provided (0.5 Mark)<br>
The value can be calculated using the following formulae,
<br><br>
$x = \dfrac{y _{int}}{2 ^{l} \, - \, 1} \quad\mathrm{and}\quad f = f _{min} \, + \, {x}\cdot{(f _{max} - f _{min})}$,<br><br>
where $y _{int}$ is the integer value of the bistring and $l$ is the length of the bitstring<br><br>
- Implement the _decodeFloatIEEE_ function, which converts the bitstring to a floating point value (0.5 Mark)
![alt text](FPReprIEEE.png "FPReprIEEEImg")
This is the binary representation of a floating point number to be considered in this function. You can assume that the length of the bitstring will always be 32 in the _decodeFloatIEEE_ function.
<br><br>
The range of the exponent will be 128 to -127, after subtracting the bias of 127 from the absolute value of the exponent.<br>
The mantissa represents the part to the right of the floating point, and the left side of the floating point is always assumed to be 1.
<br><br>
The formula, after converting the binary representations to decimal, is $\,\,f = (-1)^{sign} \times (1 \, + \, mantissa) \times 2^{\,exponent \, - \, bias}$<br>
For an example,<br>
$1\,\,10000001\,\,10110011001100110011010$<br>
$\Rightarrow (-1)^{1} \times 1.10110011001100110011010\,_{2}\times 2^{10000001 _{2} \, - \, 127}$<br>
$\Rightarrow (-1)^{1} \times 1.7000000476837158\,\times 2^{129 \, - \, 127}$<br>
$\Rightarrow -1.7000000476837158 \times 4$<br>
$\Rightarrow -6.800000190734863$

In [6]:
### 0.25 Mark
def decodeInteger(genes):
    """
    Input:  genes is an np.array of bools, i.e a bitstring
    
    Output: an integer, representing the decoded value of the bitstring
    """
    
    ### BEGIN SOLUTION
    val = 0
    base = 1
    for bit in genes[::-1]:
        val += bit * base
        base = base << 1
    
    return val
    ### END SOLUTION

In [7]:
# do not change code here
### SAMPLE TEST CASE
random.seed(221)
print("Running sample test case")
res = decodeInteger(GAString(6).genes)
ans = 58
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [8]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo(genes):
    val = 0
    base = 1
    for bit in genes[::-1]:
        val += bit * base
        base = base << 1
    
    return val

for _ in range(5):
    dude = GAString(32)
    res = decodeInteger(dude.genes)
    ans = _funkyfoo(dude.genes)
    assert ans == res

### END HIDDEN TESTS

In [9]:
### 0.5 Mark
def decodeFloatRange(genes, f_min = 0.0, f_max = 100.0):
    """
    Input:  genes is an np.array of bools, i.e a bitstring
            f_min is the lower end of the range to be mapped to
            f_max is the upper end of the range to be mapped to
    
    Output: an integer, representing the decoded value of the bitstring
    """
    
    ### BEGIN SOLUTION
    x = decodeInteger(genes)/(2**len(genes) - 1.0)
    return f_min + x * (f_max - f_min)
    ### END SOLUTION

In [10]:
# do not change code here
### SAMPLE TEST CASE
random.seed(222)
print("Running sample test case")
res = decodeFloatRange(GAString(6).genes, 23, 32)
ans = 27.285714285714285
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [11]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo1(genes):
    val = 0
    base = 1
    for bit in genes[::-1]:
        val += bit * base
        base = base << 1
    
    return val

def _funkyfoo(genes, f_min, f_max):
    x = _funkyfoo1(genes)/(2**len(genes) - 1.0)
    return f_min + x * (f_max - f_min)

for _ in range(5):
    dude = GAString(32)
    f_ = sorted(random.sample(range(2**31),2))
    res = decodeFloatRange(dude.genes,f_[0],f_[1])
    ans = _funkyfoo(dude.genes,f_[0],f_[1])
    assert np.isclose(ans,res)

### END HIDDEN TESTS

In [12]:
### 0.5 Mark
def decodeFloatIEEE(genes):
    """
    Input:  genes is an np.array of bools, i.e a bitstring
    
    Output: a floating point value, representing the decoded value of the bitstring
    """
    
    ### BEGIN SOLUTION
    exp = -127
    base = 1
    for bit in genes[1:9][::-1]:
        exp += bit * base
        base = base << 1
    
    man = 1.0
    base = 0.5
    for bit in genes[9:]:
        man += bit * base
        base = base/2
    
    return ((-1)**genes[0]) * man * (2.0**exp)
    ### END SOLUTION

In [13]:
# do not change code here
### SAMPLE TEST CASE
random.seed(223)
print("Running sample test case")
res = decodeFloatIEEE(GAString(32).genes)
ans = -3.801286575128576e+19
assert np.isclose(ans,res)
print("Sample test case passed")

Running sample test case
Sample test case passed


In [14]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo(genes):
    exp = -127
    base = 1
    for bit in genes[1:9][::-1]:
        exp += bit * base
        base = base << 1
    
    man = 1.0
    base = 0.5
    for bit in genes[9:]:
        man += bit * base
        base = base/2
    
    return ((-1)**genes[0]) * man * (2.0**exp)

for _ in range(5):
    dude = GAString(32)
    res = decodeFloatIEEE(dude.genes)
    ans = _funkyfoo(dude.genes)
    assert np.isclose(ans,res)

### END HIDDEN TESTS

### 3- Fitness functions (1 Mark)

#### 3.1 getFitness1 (0.5 Mark)
This function counts the number of 1's in the bitstring of the chromosome and returns it as the fitness.

In [15]:
### 0.5 Mark
def getFitness1(dude):
    """
    Input:  dude is a GAString object i.e. an individual chromosome
    
    Output: the fitness, as specified above in section 3.1
    """
    
    ### BEGIN SOLUTION
    return sum(dude.genes)
    ### END SOLUTION

In [16]:
# do not change code here
### SAMPLE TEST CASE
random.seed(31)
print("Running sample test case")
res = getFitness1(GAString(6))
ans = 5
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [17]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo(dude):
    return sum(dude.genes)

for _ in range(5):
    dude = GAString(32)
    assert _funkyfoo(dude) == getFitness1(dude)

### END HIDDEN TESTS

#### 3.2 getFitnessTarget (0.5 Mark)

This function converts the bitstring into an integer by treating it as a binary number and assigns a fitness score equal to the negative of the absolute difference of the chromosome and a fixed target integer.

For e.g.

101010 evaluates to 32 + 8 + 2 = 42<br>
_target_ = 69<br>
_fitness_ = - |69 - 42| = -27<br>

100101001 evaluates to 256 + 32 + 8 + 1 = 297<br>
_target_ = 420<br>
_fitness_ = - |420 - 297| = -123<br>

In [18]:
### 0.5 Mark
def getFitnessTarget(dude, target=343):
    """
    Input:  dude is a GAString object i.e. an individual chromosome
            target is the target integer
    
    Output: the fitness, as specified above in section 3.2
    """
    
    ### BEGIN SOLUTION
    fitness = 0
    base = 1
    for bit in dude.genes[::-1]:
        fitness += bit * base
        base = base << 1
    
    return -1 * abs(target - fitness)
    ### END SOLUTION

In [19]:
# do not change code here
### SAMPLE TEST CASE
random.seed(32)
print("Running sample test case")
res = getFitnessTarget(GAString(6), 27)
ans = -31
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [20]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo(dude, target):
    fitness = 0
    base = 1
    for bit in dude.genes[::-1]:
        fitness += bit * base
        base = base << 1
    
    return -1 * abs(target - fitness)

for _ in range(5):
    dude = GAString(32)
    targ = random.randint(0, 2**32 - 1)
    assert _funkyfoo(dude, targ) == getFitnessTarget(dude, targ)

### END HIDDEN TESTS

### 4- Selection (2 Marks)

Selection is a very crucial operator in a genetic algorithm. Selection rules select the individuals, called parents, that contribute to the population at the next generation. Multiple different selection operators can be used in the same iteration as well, to produce parents which prioritise different types of fitnesses. 

Some things to remember while coding -
- The GAString object has inbuilt comparators, so if the fitness for two individuals is the same, the GAString objects have to be compared directly. It is advised to use inbuilt ordering functions (such as `min`, `max`, `np.argsort` and `sorted`), for which you will have to create tuples of the form `(f(dude), dude)` for ordering the individuals correctly.
- In each function, you do not have to create new copies of the GAStrings, you will return the object(s) from the population itself.
- In each function, the input `pop` should not be changed in any way, including the ordering.

#### 4.1 Best-of Selection (0.5 Mark)
We calculate the fitnesses for the entire population and then select the top `num` candidates as our new parents, since they have a higher likelihood of sharing genes with the optimal solution. 

In [21]:
### 0.5 Mark
def selectionAll(pop, num, f):
    """
    Input:  pop is the list of all GAStrings in the current population
            num is the number of individuals to be selected
            f is the fitness function used for evaluation
    
    Output: a list of the required number of GAStrings
    
    Note: you do not have to use any random.random calls;
          the order of strings to be returned is in decreasing order of fitness
    """
    assert len(pop) > num
    
    ### BEGIN SOLUTION  
    ordered = sorted([(f(dude),dude) for dude in pop])
    parents = [dude for fit,dude in ordered[::-1][:num]]
    
    return parents
    ### END SOLUTION

In [22]:
# do not change code here
### SAMPLE TEST CASE
random.seed(41)
print("Running sample test case")
res = selectionAll(initPop(6,6), 2, getFitness1)
ans = [GAString(6, genes=np.array([0,1,1,1,0,1],dtype=bool)), GAString(6, genes=np.array([1,1,1,0,0,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
res = selectionAll(initPop(6,6), 2, getFitnessTarget)
ans = [GAString(6, genes=np.array([1,1,1,0,1,0],dtype=bool)), GAString(6, genes=np.array([1,0,1,0,1,1],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [23]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo1(dude):
    return sum(dude.genes)

def _funkyfoo2(dude, target=1596702):
    fitness = 0
    base = 1
    for bit in dude.genes[::-1]:
        fitness += bit * base
        base = base << 1
    
    return -1 * abs(target - fitness)

def _funkyfoo(pop, num, f):
    ordered = sorted([(f(dude),dude) for dude in pop])
    return [dude for fit,dude in ordered[::-1][:num]]

for _ in range(5):
    pop = initPop(32,32)
    
    res = selectionAll(pop,22,_funkyfoo1)
    ans = _funkyfoo(pop,22,_funkyfoo1)
    assert all(a == b for a,b in zip(ans,res))
    
    res = selectionAll(pop,22,_funkyfoo2)
    ans = _funkyfoo(pop,22,_funkyfoo2)
    assert all(a == b for a,b in zip(ans,res))

### END HIDDEN TESTS

#### 4.2 Randomized Selection (0.75 Mark)
Here, we multiply the fitness of each chromosome with a random number between 0 and 1, and then follow the same selection as before i.e. pick the top *num* candidates. This allows for a good chance of diversity in the parents while also ensuring the parent pool is likely to have good genes.

In [24]:
### 0.75 Mark
def selectionRand(pop, num, f):
    """
    Input:  pop is the list of all GAStrings in the current population
            num is the number of individuals to be selected
            f is the fitness function used for evaluation
    
    Output: a list of the required number of GAStrings
    
    Note: you have to use exactly len(pop) number of random.random calls, 
          do not use any other function from random
    """
    assert len(pop) > num
    
    ### BEGIN SOLUTION
    ordered = sorted([(f(dude) * random.random(), dude) for dude in pop])
    parents = [dude for fit,dude in ordered[::-1][:num]]
    
    return parents
    ### END SOLUTION

In [25]:
# do not change code here
### SAMPLE TEST CASE
random.seed(42)
print("Running sample test case")
res = selectionRand(initPop(6,6), 2, getFitness1)
ans = [GAString(6, genes=np.array([1,1,0,0,1,0],dtype=bool)), GAString(6, genes=np.array([0,1,1,1,0,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
res = selectionRand(initPop(6,6), 2, getFitnessTarget)
ans = [GAString(6, genes=np.array([1,1,1,1,1,1],dtype=bool)), GAString(6, genes=np.array([0,1,1,1,1,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [26]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo1(dude):
    return sum(dude.genes)

def _funkyfoo2(dude, target=24980923):
    fitness = 0
    base = 1
    for bit in dude.genes[::-1]:
        fitness += bit * base
        base = base << 1
    
    return -1 * abs(target - fitness)

def _funkyfoo(pop, num, f):
    ordered = sorted([(f(dude) * random.random(), dude) for dude in pop])
    return [dude for fit,dude in ordered[::-1][:num]]

for _ in range(5):
    pop = initPop(32,32)
    
    state = random.getstate()
    res = selectionRand(pop,22,_funkyfoo1)
    random.setstate(state)
    ans = _funkyfoo(pop,22,_funkyfoo1)
    assert all(a == b for a,b in zip(ans,res))
    
    state = random.getstate()
    res = selectionRand(pop,22,_funkyfoo2)
    random.setstate(state)
    ans = _funkyfoo(pop,22,_funkyfoo2)
    assert all(a == b for a,b in zip(ans,res))

### END HIDDEN TESTS

#### 4.3 Tournament Selection (0.75 Mark)

This is a more randomized form of selection, although it still uses fitness to ensure that good genes are passed down. Here, random candidates are selected for a virtual tournament, and the winner of the tournament is selected. Thus, to produce the parents, multiple tournaments are held, and the candidates for each tournament are selected independently of each other. See the image below for a simulated tournament.

![alt text](tournament_selection.png "TournamentSelectionImg")

**The tasks to be performed:**<br>

- Implement the _selectionTourney_ function, which will simulate a single tournament, from selection of candidates to deciding the champion (0.5 Mark)
- Implement the _selectionTournament_ function, which generates the required parents (0.25 Mark)

Note that since each tournament is independent, the same individual may win more than once and appear in the final selection multiple times.

In [27]:
### 0.5 Mark
def selectionTourney(pop, k, f):
    """
    Input:  pop is the list of all GAStrings in the current population
            k is the number of individuals in the tournament
            f is the fitness function used for evaluation
    
    Output: the best chromosome in the tournament
    
    Note: you have to use exactly one random.sample call, 
          do not use any other function from random
    """
    assert len(pop) > k
    
    ### BEGIN SOLUTION
    players = random.sample(range(len(pop)), k)
    (_, champ) = max((f(pop[i]), pop[i]) for i in players)
    
    return champ
    ### END SOLUTION

In [28]:
# do not change code here
### SAMPLE TEST CASE
random.seed(43)
print("Running sample test case")
res = selectionTourney(initPop(6,6), 3, getFitness1)
ans = GAString(6, genes=np.array([1,0,1,1,0,0],dtype=bool))
assert ans == res
res = selectionTourney(initPop(6,6), 3, getFitnessTarget)
ans = GAString(6, genes=np.array([1,1,1,1,0,1],dtype=bool))
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [29]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo1(dude):
    return sum(dude.genes)

def _funkyfoo2(dude, target=42973052):
    fitness = 0
    base = 1
    for bit in dude.genes[::-1]:
        fitness += bit * base
        base = base << 1
    return -1 * abs(target - fitness)

def _funkyfoo(pop, k, f):
    players = random.sample(range(len(pop)), k)
    return max((f(pop[i]), pop[i]) for i in players)[1]
    
for _ in range(5):
    pop = initPop(32,32)
    
    state = random.getstate()
    res = selectionTourney(pop,6,_funkyfoo1)
    random.setstate(state)
    ans = _funkyfoo(pop,6,_funkyfoo1)
    assert ans == res
    
    state = random.getstate()
    res = selectionTourney(pop,6,_funkyfoo2)
    random.setstate(state)
    ans = _funkyfoo(pop,6,_funkyfoo2)
    assert ans == res

### END HIDDEN TESTS

In [30]:
### 0.25 Mark
def selectionTournament(pop, num, f, k=3):
    """
    Input:  pop is the list of all GAStrings in the current population
            num is the number of individuals to be selected
            k is the number of individuals in the tournament
            f is the fitness function used for evaluation
    
    Output: a list of the required number of GAStrings
    """
    assert len(pop) > num
    
    ### BEGIN SOLUTION
    return [selectionTourney(pop, k, f) for i in range(num)]
    ### END SOLUTION

In [31]:
# do not change code here
### SAMPLE TEST CASE
random.seed(43)
print("Running sample test case")
res = selectionTournament(initPop(6,6), 2, getFitness1, 3)
ans = [GAString(6, genes=np.array([1,0,1,1,0,0],dtype=bool)), GAString(6, genes=np.array([1,1,0,1,0,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
res = selectionTournament(initPop(6,6), 2, getFitnessTarget, 3)
ans = [GAString(6, genes=np.array([1,1,0,1,0,0],dtype=bool)), GAString(6, genes=np.array([1,1,0,1,0,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [32]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo1(dude):
    return sum(dude.genes)

def _funkyfoo2(dude, target=879768743):
    fitness = 0
    base = 1
    for bit in dude.genes[::-1]:
        fitness += bit * base
        base = base << 1
    
    return -1 * abs(target - fitness)

def _funkyfoo3(pop, k, f):
    players = random.sample(range(len(pop)), k)
    return max((f(pop[i]), pop[i]) for i in players)[1]

def _funkyfoo(pop, num, f, k):
    return [_funkyfoo3(pop, k, f) for i in range(num)]

for _ in range(5):
    pop = initPop(32,32)
    
    state = random.getstate()
    res = selectionTournament(pop,22,_funkyfoo1,6)
    random.setstate(state)
    ans = _funkyfoo(pop,22,_funkyfoo1,6)
    assert all(a == b for a,b in zip(ans,res))
    
    state = random.getstate()
    res = selectionTournament(pop,22,_funkyfoo2,6)
    random.setstate(state)
    ans = _funkyfoo(pop,22,_funkyfoo2,6)
    assert all(a == b for a,b in zip(ans,res))

### END HIDDEN TESTS

### 5- Crossover (1.5 Marks) 

Crossover is the genetic operator which combines two or more parents to produce one or more offspring, for generating the population for the next iteration. How the genes are allowed to crossover is crucial to how effectively the genetic algorithm uses good intermediate solutions. In encodings which have certain validity constraints on the offspring, the crossover needs to be sophisticated so as to avoid a lot of invalid solutions, while also not prohibiting certain offspring from ever being produced. In our case, since the problem statements do not have any validity checks, we do not need to consider advanced crossover operators.

#### 5.1 Single Point Crossover (0.5 Mark)

![alt text](OnePointCrossover.svg.png "SinglePointCrossoverImg")

Randomly select a crossover point, and the two children will have the resultant spliced genes. Note that the splice point cannot be at the ends, i.e. the genes cannot be unchanged/exchanged.

In [33]:
### 0.5 Mark
def crossoverSinglePoint(dude1, dude2):
    """
    Input:  dude1 is one parent
            dude2 is the other parent
    
    Output: a list of two children, created by the one point crossover
    
    Note: you have to use exactly one random.randint call; 
          do not use any other function from random;
          you also need to create the children, do not splice the genes of the parents;
          Generate a random integer 'j' in the range [1, n-1], where n is no. of genes,
          Perform crossover/swap from indices 'j' to n-1 (both inclusive)
    """
    assert dude1.n == dude2.n
    
    ### BEGIN SOLUTION
    c1 = GAString(dude1.n, dude1.genes)
    c2 = GAString(dude2.n, dude2.genes)
    
    k = random.randint(1, dude1.n - 1)
    for i in range(k,dude1.n):
        c1.genes[i], c2.genes[i] = c2.genes[i], c1.genes[i]
    
    return [c1,c2]
    ### END SOLUTION

In [34]:
# do not change code here
### SAMPLE TEST CASE
random.seed(51)
print("Running sample test case")
res = crossoverSinglePoint(GAString(6),GAString(6))
ans = [GAString(6, genes=np.array([1,0,1,0,1,0],dtype=bool)), GAString(6, genes=np.array([0,1,0,0,0,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [35]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo(dude1, dude2):
    k = random.randint(1, dude1.n - 1)
    c1 = GAString(dude1.n, dude1.genes)
    c2 = GAString(dude2.n, dude2.genes)
    for i in range(k,dude1.n):
        c1.genes[i], c2.genes[i] = c2.genes[i], c1.genes[i]
    return [c1,c2]

for _ in range(5):
    dude1 = GAString(32)
    dude2 = GAString(32)
    
    state = random.getstate()
    res = crossoverSinglePoint(dude1,dude2)
    random.setstate(state)
    ans = _funkyfoo(dude1,dude2)
    assert all(a == b for a,b in zip(ans,res))

### END HIDDEN TESTS

#### 5.2 Two Point Crossover (0.5 Mark)

![alt text](TwoPointCrossover.svg.png "TwoPointCrossoverImg")

Randomly select two crossover points, and the two children will have the resultant spliced genes in between the two points.<br>
Note that for our implementation, the crossover points will never be at the ends i.e. the two point crossover operator can never produce results like the one point crossover operator, or vice versa. The points cannot coincide as well, so the children should never get unchanged genes.

In [36]:
### 0.5 Mark
def crossoverTwoPoint(dude1, dude2):
    """
    Input:  dude1 is one parent
            dude2 is the other parent
    
    Output: a list of two children, created by the two point crossover
    
    Note: you have to use exactly one random.sample call;
          do not use any other function from random;
          you also need to create the children, do not splice the genes of the parents;
          Generate random integers j1 and j2 (s.t. j1 < j2) in the range [1, n-1], where n is the no. of genes,
          Perform crossover from indices j1 to j2-1 (both inclusive)
    """
    assert dude1.n == dude2.n
    
    ### BEGIN SOLUTION
    c1 = GAString(dude1.n, dude1.genes)
    c2 = GAString(dude2.n, dude2.genes)
    
    k1,k2 = sorted(random.sample(range(1,dude1.n), 2))
    for i in range(k1,k2):
        c1.genes[i], c2.genes[i] = c2.genes[i], c1.genes[i]
    
    return [c1,c2]
    ### END SOLUTION

In [37]:
# do not change code here
### SAMPLE TEST CASE
random.seed(52)
print("Running sample test case")
res = crossoverTwoPoint(GAString(6),GAString(6))
ans = [GAString(6, genes=np.array([0,1,0,1,1,1],dtype=bool)), GAString(6, genes=np.array([0,1,0,1,0,1],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [38]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo(dude1, dude2):
    k1,k2 = sorted(random.sample(range(1,dude1.n), 2))
    c1 = GAString(dude1.n, dude1.genes)
    c2 = GAString(dude2.n, dude2.genes)
    for i in range(k1,k2):
        c1.genes[i], c2.genes[i] = c2.genes[i], c1.genes[i]
    return [c1,c2]

for _ in range(5):
    dude1 = GAString(32)
    dude2 = GAString(32)
    
    state = random.getstate()
    res = crossoverTwoPoint(dude1,dude2)
    random.setstate(state)
    ans = _funkyfoo(dude1,dude2)
    assert all(a == b for a,b in zip(ans,res))

### END HIDDEN TESTS

#### 5.3 Uniform Crossover (0.5 Mark)

![alt text](UniformCrossover.png "UniformCrossoverImg")

Here, the children have equal probability of inheriting a specific gene from either parent, i.e. each gene is independently passed on to either child randomly.

In [39]:
### 0.5 Mark
def crossoverUniform(dude1, dude2):
    """
    Input:  dude1 is one parent
            dude2 is the other parent
    
    Output: a list of two children, created by the uniform crossover
    
    Note: you have to use exactly dude1.n number of random.random calls; 
          do not use any other function from random;
          you also need to create the children, do not splice the genes of the parents;
          For index j in dude1.genes, if random.random() < 0.5, then assign dude1.genes[j] to c2.genes[j],
              else assign dude1.genes[j] to c1.genes[j]
    """
    assert dude1.n == dude2.n
    
    ### BEGIN SOLUTION
    c1 = GAString(dude1.n, dude1.genes)
    c2 = GAString(dude2.n, dude2.genes)
    
    for i in range(dude1.n):
        if random.random() < 0.5:
            c1.genes[i], c2.genes[i] = c2.genes[i], c1.genes[i]
    
    return [c1,c2]
    ### END SOLUTION

In [40]:
# do not change code here
### SAMPLE TEST CASE
random.seed(53)
print("Running sample test case")
res = crossoverUniform(GAString(6),GAString(6))
ans = [GAString(6, genes=np.array([0,0,1,1,1,0],dtype=bool)), GAString(6, genes=np.array([0,0,0,0,0,0],dtype=bool))]
assert all(a == b for a,b in zip(ans,res))
print("Sample test case passed")

Running sample test case
Sample test case passed


In [41]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo(dude1, dude2):
    c1 = GAString(dude1.n, dude1.genes)
    c2 = GAString(dude2.n, dude2.genes)
    for i in range(dude1.n):
        if random.random() < 0.5:
            c1.genes[i], c2.genes[i] = c2.genes[i], c1.genes[i]
    return [c1,c2]

for _ in range(5):
    dude1 = GAString(32)
    dude2 = GAString(32)
    
    state = random.getstate()
    res = crossoverUniform(dude1,dude2)
    random.setstate(state)
    ans = _funkyfoo(dude1,dude2)
    assert all(a == b for a,b in zip(ans,res))

### END HIDDEN TESTS

### 6- Mutation (1.75 Marks)

Mutation is a genetic operator used to maintain genetic diversity from one generation of a population of genetic algorithm chromosomes to the next. There are many different types of mutation operators as well - single point, uniform, inversion, swap etc. Many mutation operators work on encoding with integers and floating point numbers as well.

#### 6.1 Single Point Mutation (0.5 Mark)

![alt text](SinglePointMutation.png "SinglePointMutationImg")

A random bit is selected and flipped. This type of mutation is used early in the algorithm to ensure diversity in solutions.

In [42]:
### 0.5 Mark
def mutationSinglePoint(dude):
    """
    Input:  dude is the original chromosome
    
    Output: a new chromosome, with one bit flipped
    
    Note: you have to use exactly one random.randint call, 
          do not use any other function from random;
          you also need to create the mutated chromosome, do not mutate the genes of the original;
          Generate a random integer 'j' in the range [0, n-1], where n is no. of genes, and flip genes[j]
    """
    
    ### BEGIN SOLUTION
    b = random.randint(0, dude.n - 1)
    
    dudeX = GAString(dude.n,dude.genes)
    dudeX.genes[b] = not dudeX.genes[b]
    
    return dudeX
    ### END SOLUTION

In [43]:
# do not change code here
### SAMPLE TEST CASE
random.seed(61)
print("Running sample test case")
res = mutationSinglePoint(GAString(6))
ans = GAString(6, genes=np.array([1,0,0,0,1,0],dtype=bool))
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [44]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo(dude):
    b = random.randint(0, dude.n - 1)
    dudeX = GAString(dude.n,dude.genes)
    dudeX.genes[b] = not dudeX.genes[b]
    return dudeX

for _ in range(5):
    dude = GAString(32)
    
    state = random.getstate()
    res = mutationSinglePoint(dude)
    random.setstate(state)
    ans = _funkyfoo(dude)
    assert ans == res

### END HIDDEN TESTS

#### 6.2 Uniform Mutation (0.5 Mark)

![alt text](UniformMutation.png "UniformMutationImg")

Mutation occurs during evolution according to a user-definable mutation probability. For each bit, the bit is flipped if a random number generated is less than the probability specified. This probability should be set low. If it is set too high, the algorithm will turn into a primitive random search. 

In [45]:
### 0.5 Mark
def mutationUniform(dude, prob=0.05):
    """
    Input:  dude is the original chromosome
            prob is the probability of flipping an individual bit
    
    Output: a new chromosome, after applying uniform mutation
    
    Note: you have to use exactly dude.n number of random.random call;
          do not use any other function from random;
          you also need to create the mutated chromosome, do not mutate the genes of the original;
          For each index j in dude.genes, mutate it if random.random() < prob.
    """
    assert 0 <= prob <= 1
    
    ### BEGIN SOLUTION
    dudeX = GAString(dude.n,dude.genes)
    
    for i in range(dude.n):
        if random.random() < prob:
            dudeX.genes[i] = not dudeX.genes[i]
    
    return dudeX
    ### END SOLUTION

In [46]:
# do not change code here
### SAMPLE TEST CASE
random.seed(62)
print("Running sample test case")
res = mutationUniform(GAString(6),0.04)
ans = GAString(6, genes=np.array([0,1,0,1,1,0],dtype=bool))
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [47]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo(dude,prob=0.05):
    dudeX = GAString(dude.n,dude.genes)
    for i in range(dude.n):
        if random.random() < prob:
            dudeX.genes[i] = not dudeX.genes[i]
    return dudeX

for _ in range(5):
    dude = GAString(32)
    
    state = random.getstate()
    res = mutationUniform(dude,0.06)
    random.setstate(state)
    ans = _funkyfoo(dude,0.06)
    assert ans == res

### END HIDDEN TESTS

#### 6.3 Inversion Mutation (0.75 Mark)

![alt text](InversionMutation.png "InversionMutationImg")

Select two random points in the chromosome, and invert the bits in between. See the picture above for clarity. Note that the entire chromosome can be inverted as well. 

In [48]:
### 0.75 Mark
def mutationInversion(dude):
    """
    Input:  dude is the original chromosome
    
    Output: a new chromosome, after applying inversion mutation
    
    Note: you have to use exactly one random.sample call; 
          the range generated has to be inverted inclusive of the ends;
          do not use any other function from random;
          you also need to create the mutated chromosome, do not mutate the genes of the original;
          Generate random integers j1 and j2 (s.t. j1 < j2) in the range [0, n-1], where n is the no. of genes,
          Invert the bits from indices j1 to j2, including the ends
    """
    ### BEGIN SOLUTION
    dudeX = GAString(dude.n,dude.genes)
    
    k1, k2 = sorted(random.sample(range(0,dude.n), 2))
    for i in range((k2-k1+1)//2):
        dudeX.genes[k1+i], dudeX.genes[k2-i] = dudeX.genes[k2-i], dudeX.genes[k1+i]
    
    return dudeX
    ### END SOLUTION

In [49]:
# do not change code here
### SAMPLE TEST CASE
random.seed(63)
print("Running sample test case")
res = mutationInversion(GAString(7))
ans = GAString(7, genes=np.array([1,0,1,1,1,0,1],dtype=bool))
assert ans == res
print("Sample test case passed")

Running sample test case
Sample test case passed


In [50]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo(dude):
    k1,k2 = sorted(random.sample(range(0,dude.n), 2))
    dudeX = GAString(dude.n,dude.genes)
    for i in range((k2-k1+1)//2):
        dudeX.genes[k1+i], dudeX.genes[k2-i] = dudeX.genes[k2-i], dudeX.genes[k1+i]
    return dudeX

for _ in range(1):
    dude = GAString(8)
    
    state = random.getstate()
    res = mutationInversion(dude)
    random.setstate(state)
    ans = _funkyfoo(dude)
    assert ans == res

### END HIDDEN TESTS

### Single Iteration (1 Mark)

Finally, we have all the tools needed for a single iteration of a genetic algorithm. Now to actually implement the entire thing. Given below is a template for the _singleIteration_ function, where we can change everything, from the sizes of the elite group to the mutation operator itself,

Steps to be followed for constructing the new population - 
1. Elitism - Select the best `SIZE_ELITE` individuals to be passed on directly to the next generation. Note that these individuals will also be considered in selection in the same iteration as well.
2. Selection - Parents will be chosen according to the given _selection_ function, no assumptions are to be made about the function. Treat it as a black box, with the same function signature as _selectionAll_ and _selectionRand_ i.e. the function will be called as `selection(pop, num, f)`.
3. Crossover - New individuals will be created by treating the _crossover_ function as a black box as well. The call to be made is `crossover(dude1,dude2)`. Use consecutive individuals from the list of parents (obtained after performing selection) for each call, and do not reuse any parents. Note that it is not necessary that the crossover will return exactly 2 individuals, however it can be assumed that a list of one or more individuals will be returned.  
4. Mutation - First of all, we will select `SIZE_POP - SIZE_ELITE - SIZE_SELECTION` worst individuals from original population (i.e. `pop`) according to the fitness function i.e. the individuals will be selected on the basis of negative of their fitness values. The same selection operator as used previously in step 2 will be applied here. Each such selected individual will be subject to mutation and its mutated version will be present in the next generation. The call which is to be made for mutation will be `mutation(dude)` [treated as a black box] and its output will be a single new individual.
5. New population - Here, you just need to combine all the results from the previous steps i.e. elites + crossover children + mutated individuals.

You may have observed that since crossover is not guaranteed to produce the same number of individuals as from selection, the size of the population can decrease if we wish it to. This technique is sometimes used in conjunction with generating entirely new individuals to inject some fresh genes into the population in the later stages of the algorithm.

A helper function, _getBestDude,_ has been provided to check for the best individual in a population at any time. It returns both the fitness and the individual.

**NOTE**: Do not attempt this question before implementing atleast one of the functions in each of the previous sections successfully. Sample test cases have not been provided as well. If you wish to test the function for yourself, a playground cell has been provided below.

In [51]:
# do not change code here
def getBestDude(pop, f):
    return max((f(dude),dude) for dude in pop)

In [52]:
### 1 Mark
def singleIteration(pop,
                    f,
                    SIZE_POP,
                    SIZE_ELITE,
                    SIZE_SELECTION,
                    selection=selectionRand,
                    crossover=crossoverUniform,
                    mutation=mutationUniform):
    """
    Input:  pop is the list of all GAStrings in the current population
            f is the fitness function used for evaluation
            SIZE_POP is the number of individuals in the population
            SIZE_ELITE is the number of elite individuals passed on directly
            SIZE_SELECTION is number of parents to be selected
            selection is the selection function to be used
            crossover is the crossover function to be used
            mutation is the mutation function to be used
    
    Output: a new population, after applying elitism, selection, crossover and mutation;
            in terms of code specs, a list of GAStrings
    
    Note: do not use any random package function calls here directly,
          the random calls should exist within the genetic operator functions only
    """
    
    ### BEGIN SOLUTION
    elites = selectionAll(pop, SIZE_ELITE, f)
    
    parents = selection(pop, SIZE_SELECTION, f)
    children = []
    for i in range(0,SIZE_SELECTION,2):
        children.extend(crossover(parents[i],parents[i+1]))
    
    f_ = lambda x: -f(x)
    experiments = selection(pop, SIZE_POP - SIZE_ELITE - SIZE_SELECTION, f_)
    mutants = [mutation(dude) for dude in experiments]
    
    newpop = elites + children + mutants
    
    return newpop
    ### END SOLUTION

In [53]:
# Playground
# You can use this as a sample test case, 
# although note that the assert check is only for the best individual in each iteration
random.seed(69)
print("Running sample test case")
pop = initPop(20,32)
f = getFitness1
s0 = 20
s1 = 4
s2 = 16
s = selectionRand
c = crossoverUniform
m = mutationInversion

res = []
ans = [GAString(32, genes=np.array([1,1,1,1,1,0,1,1,1,0,1,1,0,0,1,1,0,0,0,1,1,0,1,1,1,0,1,1,0,1,1,1],dtype=bool)),
       GAString(32, genes=np.array([1,0,1,1,1,0,1,1,1,1,1,1,0,1,1,1,0,0,0,1,1,0,1,0,1,1,1,0,1,1,1,1],dtype=bool)),
       GAString(32, genes=np.array([1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,0,1,0,0],dtype=bool)),
       GAString(32, genes=np.array([1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,0,1,0,0],dtype=bool)),
       GAString(32, genes=np.array([1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,1,1,0,0,1,1,1,1,1,1,1,1,1,0,1,0,0],dtype=bool))]

for i in range(5):
    pop = singleIteration(pop,f,s0,s1,s2,s,c,m)
    res.append(getBestDude(pop,f)[1])
    assert res[i] == ans[i]
print("Sample test case passed")

Running sample test case
Sample test case passed


In [54]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfooINIT(size_pop, number_of_genes):
    return [GAString(number_of_genes) for _ in range(size_pop)]

def _funkyfooF1(dude):
    return sum(dude.genes)

def _funkyfooF2(dude, target=139284925):
    fitness = 0
    base = 1
    for bit in dude.genes[::-1]:
        fitness += bit * base
        base = base << 1
    return -1 * abs(target - fitness)

def _funkyfooS1(pop, num, f):
    ordered = sorted([(f(dude), dude) for dude in pop])
    return [dude for fit,dude in ordered[::-1][:num]]

def _funkyfooS2(pop, num, f):
    ordered = sorted([(f(dude) * random.random(), dude) for dude in pop])
    return [dude for fit,dude in ordered[::-1][:num]]

def _funkyfooS3_(pop, k, f):
    players = random.sample(range(len(pop)), k)
    return max((f(pop[i]), pop[i]) for i in players)[1]

def _funkyfooS3(pop, num, f, k):
    return [_funkyfooS3_(pop, k, f) for i in range(num)]

def _funkyfooC1(dude1, dude2):
    k = int(random.random() * dude1.n)
    c1 = GAString(dude1.n, dude1.genes)
    c2 = GAString(dude2.n, dude2.genes)
    for i in range(k,dude1.n):
        c1.genes[i], c2.genes[i] = c2.genes[i], c1.genes[i]
    return [c1,c2]

def _funkyfooC2(dude1, dude2):
    k1,k2 = sorted(random.sample(range(1,dude1.n), 2))
    c1 = GAString(dude1.n, dude1.genes)
    c2 = GAString(dude2.n, dude2.genes)
    for i in range(k1,k2):
        c1.genes[i], c2.genes[i] = c2.genes[i], c1.genes[i]
    return [c1,c2]

def _funkyfooC3(dude1, dude2):
    c1 = GAString(dude1.n, dude1.genes)
    c2 = GAString(dude2.n, dude2.genes)
    for i in range(dude1.n):
        if random.random() < 0.5:
            c1.genes[i], c2.genes[i] = c2.genes[i], c1.genes[i]
    return [c1,c2]

def _funkyfooM1(dude):
    b = random.randint(0, dude.n - 1)
    dudeX = GAString(dude.n,dude.genes)
    dudeX.genes[b] = not dudeX.genes[b]
    return dudeX

def _funkyfooM2(dude,prob=0.06):
    dudeX = GAString(dude.n,dude.genes)
    for i in range(dude.n):
        if random.random() < prob:
            dudeX.genes[i] = not dudeX.genes[i]
    return dudeX

def _funkyfooM3(dude):
    k1,k2 = sorted(random.sample(range(0,dude.n), 2))
    dudeX = GAString(dude.n,dude.genes)
    for i in range((k2-k1+1)//2):
        dudeX.genes[k1+i], dudeX.genes[k2-i] = dudeX.genes[k2-i], dudeX.genes[k1+i]
    return dudeX

def _funkyfoo(pop,
              f,
              SIZE_POP,
              SIZE_ELITE,
              SIZE_SELECTION,
              selection,
              crossover,
              mutation):
    
    elites = selectionAll(pop, SIZE_ELITE, f)
    
    parents = selection(pop, SIZE_SELECTION, f)
    children = []
    for i in range(0,SIZE_SELECTION,2):
        children.extend(crossover(parents[i],parents[i+1]))
    
    f_ = lambda x: -f(x)
    experiments = selection(pop, SIZE_POP - SIZE_ELITE - SIZE_SELECTION, f_)
    mutants = [mutation(dude) for dude in experiments]
    
    newpop = elites + children + mutants
    
    return newpop

pop = initPop(50,32)
f = _funkyfooF2
s0 = 50
s1 = 2
s2 = 44
s = _funkyfooS2
c = _funkyfooC3
m = _funkyfooM3

for _ in range(20):
    fit,dude = getBestDude(pop,f)
    print(f"Chromosome = {dude.genes.astype(int)} | Fitness = {fit}")
    state = random.getstate()
    res = singleIteration(pop,f,s0,s1,s2,s,c,m)
    random.setstate(state)
    ans = _funkyfoo(pop,f,s0,s1,s2,s,c,m)
    assert all(a == b for a,b in zip(ans,res))
    pop = ans

fit,dude = getBestDude(pop,f)
print(f"Chromosome = {dude.genes.astype(int)} | Fitness = {fit}")
    
### END HIDDEN TESTS

Chromosome = [0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 1 1 1 1 1 1 1 1 0 0 1 0 0 1 1 1 1] | Fitness = -17083538
Chromosome = [0 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 1] | Fitness = -2017418
Chromosome = [0 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 1] | Fitness = -2017418
Chromosome = [0 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 1] | Fitness = -2017418
Chromosome = [0 0 0 0 1 0 0 0 0 1 1 0 1 1 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 1 1] | Fitness = -2017418
Chromosome = [0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 0 0 0 1 1 1 1] | Fitness = -130386
Chromosome = [0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 0 0 0 1 1 1 1] | Fitness = -130386
Chromosome = [0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 0 1 1 1 1] | Fitness = -114066
Chromosome = [0 0 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 1 0 0 1 1 0 1] | Fitness = -114064
Chromosome = [0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 0 1 1 1 0 0 0 1 0 0 0 1 1 0] | Fitness = -13687
Chrom

Well, that was anticlimactic, wasn't it? Is that really all it takes to solve complicated problems? Just changing the fitness evaluator and providing your own encoding/decoding functions?

Well, not quite. You need to tune the genetic algorithm as well. Mutation rate, choice of operators, writing your own mutation and crossover operators, applying mutation on only the worst individuals, generating new individuals every once in a while - there's lots of choice on what to do, and it becomes very important to decide what not to do as well.

As an exercise (non-graded, promise), try tuning the mutation rate, elitism ratio and number of generations and see what the effect on convergence is.

# PID Control Design (5 marks)

The assignment would be helpful to build the intuition behind controller design. Various prompts will be given throughout the assignment with the intention to ponder and understand the implication of each step. The assignment has been developed to stimulate critical thinking and think like engineers. The prompts are ungraded but students are encouraged to discuss these prompts.


### Terminologies used

$K_p$ denotes the proportional gain

$K_i$ denotes the integral gain

$K_d$ denotes the derivative gain

$H(s)$ denotes the transfer function in the **s** domain


### Basic Theory
As discussed in the Workshop, any system can be modeled as shown below. 

<center><img src="pid_diagram.JPG" width="512" height="200"></center>

```Prompt: What are the limitation of such models? Think in terms of accuracy and feasibility```

One of the most widely used controller is the *PID controller*, which stands for Proportional, Integral, Derivative Controller. The PID Control output is defined as

$m(t) = K_p e(t) + K_i \int e(t) dt + K_d \frac{d e(t)}{dt} $ 

```Prompt: Can PID controller be used for all applications? Think in term of linearity```

Normally the system dynamics is represented in terms of its Laplacian. We convert the diferential equation in time domain (t) to its Laplacian domain (s) 

$e(t)$ -> $E(s)$

$\int e(t) dt$ -> $\frac{E(s)}{s}$

$\frac{d e(t)}{dt}$ -> $s E(s)$

Till now we have been dealing with continuous variables (t,s). Unfortunately, computers are incapable of analog simulations and have to be discretized.

Discretizing a continuous signal $x(t)$ to discrete sequence $x[n]$ simply involves sampling. By keeping a high sampling frequency, we can effectively imitate analog simulations.

$x[n] = x(nT_s)$ where $T_s$ is the sampling period

<center><img src="sampling.jpg" width="512" height="200"></center>

```Prompt: What happens if the sampling frequency is too low? How about when it is too high?```

### 7- Plant Design

As explained earlier, the plant model can be expressed in terms of a transfer function in the s domain. 
The transfer function is basically a relation of the output in terms of the input. 

#### 7.1 Designing the Input (1 Mark)
Control engineers test a system by giving it some standard inputs and see its response. 

**The tasks to be performed:**<br>
  Design various inputs
- Impulse (0.25 Mark)
- Step (0.25 Mark)
- Ramp (0.5 Mark)

<center><img src="Impulse.JPG" width="256" height="100"></center>
<center>Impulse Response at $t_d$ at 1 sec and magnitude = 1</center>
<center><img src="Step.JPG" width="256" height="100"></center>
<center>Step Response at $t_d$ at 1 sec and magnitude = 1</center>
<center><img src="Ramp.JPG" width="256" height="100"></center>
<center>Ramp Response at $t_d$ at 1 sec and slope =  1</center>

In [55]:
# do not change code here
Fs = 1000 # Sampling Frequency
Ti = 0 # Initial Time of simulation
Tf = 5 # Final Time of simulation
Ts = 1 / Fs # Sampling Period

In [56]:
### 0.25 Marks
def impulse(Ti, Tf,Fs,td,A):
    """
    Inputs:
        td : The time where the impulse occurs
        A: Amplitude of the impulse
    Outputs:
        Impulse response. (numpy array) Note: that it must be in discrete domain and a discrete impulse
                                        signal has a finite height given by A.
                                        Length: floor((Tf- Ti)*Fs)
    
    """
    ### BEGIN SOLUTION
    Ts = 1/Fs
    n = math.floor((Tf- Ti)*Fs)
    imp = np.zeros(n)
    n0 = math.floor((td-Ti)*Fs)
    imp[n0] = A
    return imp
    ### END SOLUTION

In [57]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
sol_im = np.load('impulse_response.npy')
imp = impulse(Ti,Tf,Fs,0,1)

assert np.allclose(sol_im, imp)
print("Sample test case passed")

Running sample test case


FileNotFoundError: [Errno 2] No such file or directory: 'impulse_response.npy'

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS
def sol_impulse(Ti, Tf,Fs,td,A):
    Ts = 1/Fs
    n = math.floor((Tf- Ti)*Fs)
    imp = np.zeros(n)
    n0 = math.floor((td-Ti)*Fs)
    imp[n0] = A
    return imp
sol_imp = sol_impulse(Ti,Tf,Fs,2,5)
imp = impulse(Ti,Tf,Fs,2,5)

assert np.allclose(sol_imp, imp)
### END HIDDEN TESTS

In [ ]:
### 0.25 Marks
def step(Ti, Tf,Fs,td,A):
    """
    Inputs:
        td : The time where the step starts occurs
        A: Amplitude of the step response
    Outputs:
        Step response. (numpy array) Note that it must be in discrete domain
                                     Length: floor((Tf- Ti)*Fs)
    
    """
    ### BEGIN SOLUTION
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    imp = np.zeros(n)
    n0 = math.floor((td-Ti)*Fs)
    imp[n0:] = A
    return imp
    ### END SOLUTION

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
sol_im = np.load('step_response.npy')
imp = step(Ti,Tf,Fs,0,1)

assert np.allclose(sol_im, imp)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS
def sol_step(Ti, Tf,Fs,td,A):
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    imp = np.zeros(n)
    n0 = math.floor((td-Ti)*Fs)
    imp[n0:] = A
    return imp

sol_rr = sol_step(Ti,Tf,Fs,2,2)
rr = step(Ti,Tf,Fs,2,2)

assert np.allclose(sol_rr, rr)
### END HIDDEN TESTS

In [ ]:
### 0.5 Marks
def ramp(Ti, Tf,Fs,td,K):
    """
    Inputs:
        td : The time where the rmp starts occurs
        K: Rate of increase of ramp
    Outputs:
        Ramp response. (numpy array) Note that it must be in discrete domain
                                     Length: floor((Tf- Ti)*Fs)
    
    """
    ### BEGIN SOLUTION
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    imp = np.zeros(n)
    n0 = math.floor((td-Ti)*Fs)
    nf = Tf*Fs
    imp[n0:] = K*Ts *np.arange(0,(nf-n0))
    return imp
    ### END SOLUTION

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
sol_im = np.load('ramp_response.npy')
imp = ramp(Ti,Tf,Fs,0,1)

assert np.allclose(sol_im, imp)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS
def sol_ramp(Ti, Tf,Fs,td,K):
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    imp = np.zeros(n)
    n0 = math.floor((td-Ti)*Fs)
    nf = Tf*Fs
    imp[n0:] = K*Ts *np.arange(0,(nf-n0))
    return imp

sol_rr = sol_ramp(Ti,Tf,Fs,2,2)
rr = ramp(Ti,Tf,Fs,2,2)

assert np.allclose(sol_rr, rr)
### END HIDDEN TESTS

#### 7.2 Designing the Plant (0.5 Mark)
As discussed earlier, the plant is a often described by its transfer function. A transfer function is describes the relation of the output with respect to the input. 

Assume a second order system. With a general form:
$H(s) = \frac{Output(s)}{Input(s)} = \frac{K}{1 + \tau _p s}$, Where $\tau_p$ is the plant time constant

Taking the input as $x(t)$ and output as $y(t)$,

$H(s) = \frac{Y(s)}{X(s)} = \frac{K}{1 + \tau _p s}$

$Y(s) + \tau _p s Y(s) = K X(s)$

Which leads to:

$\tau_p \frac{dy(t)}{dt} + y(t) = K x(t)$

```Prompt: What is the effect of increasing Tau? Think in perspective of latency of system```

In [ ]:
### 0.5 Marks
def plant(K,tau_p,Ti,Tf,Fs,plant_input):
    """
    Inputs:
        K : The time where the rmp starts occurs
        tau_p: Time constant of the plant
        Ti: Initial time
        Tf: Final Time
        Fs: Sampling frequency
        plant_input: Input to the plant (numpy array)
    Outputs:
        Plant Dyanmics. (numpy array) Note that it must be in discrete domain
                                      Length: floor((Tf- Ti)*Fs)
    
    """
    ### BEGIN SOLUTION
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    response = np.zeros(n+1)
    for i in range(0,n):
        response[i+1] = (1 - (Ts/tau_p))*response[i] + (K*Ts/tau_p)*plant_input[i]
    response = response[1:]
    return response
    ### END SOLUTION

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
sol_dynamics= np.load('plant_dynamics.npy')
plant_input = np.load('plant_input.npy')
dynamics = plant(1,2,Ti,Tf,Fs,plant_input)
assert np.allclose(sol_dynamics, dynamics)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS
def sol_plant(K,tau_p,Ti,Tf,Fs,plant_input):
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    response = np.zeros(n+1)
    for i in range(0,n):
        response[i+1] = (1 - (Ts/tau_p))*response[i] + (K*Ts/tau_p)*plant_input[i]
    return response[1:]

n = math.floor(Tf / Ts)
sol_dynamics = sol_plant(5,2,Ti,Tf,Fs,2*np.ones(n+1))
dynamics = plant(5,2,Ti,Tf,Fs,2*np.ones(n+1))

assert np.allclose(sol_dynamics, dynamics)
### END HIDDEN TESTS

### 8- PID Control (2.5 Marks)
PID controllers are designed using two methods, using the position form or via its velocity form. The position form gives the output of the PID controller at that given instant whereas the velocity form implementation gives the change in PID controller output from its previous value.

Assume a first order plant (as modelled earlier)

**The tasks to be performed:**<br>

- Position Form Implementation of PID Controller (1 Mark)
- Velocity Form Implementation of PID Controller (1 Mark)

Hint: For the velocity form - Write the discrete equation at $n^{th}$ instant and $(n-1)^{th}$ instant

```Prompt: What is the advantage/disadvantage of the two implementations? Which form is preferred between position and velocity form?```

In [ ]:
### 1 Mark
def pid_pos(Kp, Ki, Kd,reference, K, tau_p,Ti,Tf,Fs):
    """
    Inputs:
        Kp: Proportional gain
        Ki: Integral gain
        Kp: Derivative gain
        reference: Reference of the system (numpy array) Length: floor((Tf- Ti)*Fs)+1
        K: Gain of plant
        tau_p: Time constant of the plant
        Ti: Initial time
        Tf: Final Time
        Fs: Sampling frequency
    Outputs:
        Plant Response. (numpy array) Note that it must be in discrete domain
                                      Length: floor((Tf- Ti)*Fs)
    
    """
    ### BEGIN SOLUTION
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    m_cont = np.zeros(n+1)
    y = np.zeros(n+1)
    e = np.zeros(n+1)
    int_e = np.zeros(n+1)
    diff_e = np.zeros(n+1)
    
    for i in range(0,n):
        y[i+1] = (1 - (Ts/tau_p))*y[i] + (K*Ts/tau_p)*m_cont[i]
        e[i+1] = reference[i+1] - y[i+1] # Calculation of Error
        int_e[i+1] = int_e[i] + Ts* e[i+1] # Calculation of Integral of Error
        diff_e[i+1] = (e[i+1] - e[i]) /Ts # Calculation of Derivative of Error
        m_cont[i+1] = Kp* e[i+1] + Ki * int_e[i+1] + Kd* diff_e[i+1] # Calculation of Controller Output
    y = y[1:]
    return y
    ### END SOLUTION

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
pid_pos_dynamics= np.load('pid_pos_dynamics.npy')
reference = np.load('reference.npy')
dynamics = pid_pos(5, 5, 0.1,reference, 1, 2,Ti,Tf,Fs)

assert np.allclose(pid_pos_dynamics, dynamics)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS
def sol_pid_pos(Kp, Ki, Kd,reference, K, tau_p,Ti,Tf,Fs):
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    m_cont = np.zeros(n+1)
    y = np.zeros(n+1)
    e = np.zeros(n+1)
    int_e = np.zeros(n+1)
    diff_e = np.zeros(n+1)
    
    for i in range(0,n):
        y[i+1] = (1 - (Ts/tau_p))*y[i] + (K*Ts/tau_p)*m_cont[i]
        e[i+1] = reference[i+1] - y[i+1] # Calculation of Error
        int_e[i+1] = int_e[i] + Ts* e[i+1] # Calculation of Integral of Error
        diff_e[i+1] = (e[i+1] - e[i]) /Ts # Calculation of Derivative of Error
        m_cont[i+1] = Kp* e[i+1] + Ki * int_e[i+1] + Kd* diff_e[i+1] # Calculation of Controller Output
    return y[1:]

n = math.floor(Tf / Ts)
sol_dynamics = sol_pid_pos(2.5, 5, 0.1,2*np.ones(n+1), 2, 2,Ti,Tf,Fs)
dynamics = pid_pos(2.5, 5, 0.1,2*np.ones(n+1), 2, 2,Ti,Tf,Fs)

assert np.allclose(sol_dynamics, dynamics)
### END HIDDEN TESTS

In [ ]:
### 1 Mark
def pid_velocity(Kp, Ki, Kd,reference, K, tau_p,Ti,Tf,Fs):
    """
    Inputs:
        Kp: Proportional gain
        Ki: Integral gain
        Kp: Derivative gain
        reference: Reference of the system (numpy array) Length: floor((Tf- Ti)*Fs)+1
        K: Gain of plant
        tau_p: Time constant of the plant
        Ti: Initial time
        Tf: Final Time
        Fs: Sampling frequency
    Outputs:
        Plant Dyanmics. (numpy array) Note that it must be in discrete domain
                                      Length: floor((Tf- Ti)*Fs)
    
    """
    ### BEGIN SOLUTION
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    m_cont = np.zeros(n+1)
    y = np.zeros(n+1)
    e = np.zeros(n+1)
    int_e = np.zeros(n+1)
    diff_e = np.zeros(n+1)
    
    for i in range(0,n):
        y[i+1] = (1 - (Ts/tau_p))*y[i] + (K*Ts/tau_p)*m_cont[i]
        e[i+1] = reference[i+1] - y[i+1] # Calculation of Error
        int_e[i+1] = int_e[i] + Ts* e[i+1] # Calculation of Integral of Error
        diff_e[i+1] = (e[i+1] - e[i]) /Ts # Calculation of Derivative of Error
        m_cont[i+1] = m_cont[i] + Ts*(Kp* diff_e[i+1] + Ki * e[i+1] + (Kd/Ts)* (diff_e[i+1]-diff_e[i])) # Calculation of Controller Output
    y = y[1:]
    return y
    ### END SOLUTION

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
pid_velocity_dynamics= np.load('pid_velocity_dynamics.npy')
reference = np.load('reference.npy')
dynamics = pid_velocity(5, 5, 0.1,reference, 1, 2,Ti,Tf,Fs)
assert np.allclose(pid_velocity_dynamics, dynamics)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS
def sol_pid_velocity(Kp, Ki, Kd,reference, K, tau_p,Ti,Tf,Fs):
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    m_cont = np.zeros(n+1)
    y = np.zeros(n+1)
    e = np.zeros(n+1)
    int_e = np.zeros(n+1)
    diff_e = np.zeros(n+1)
    
    for i in range(0,n):
        y[i+1] = (1 - (Ts/tau_p))*y[i] + (K*Ts/tau_p)*m_cont[i]
        e[i+1] = reference[i+1] - y[i+1] # Calculation of Error
        int_e[i+1] = int_e[i] + Ts* e[i+1] # Calculation of Integral of Error
        diff_e[i+1] = (e[i+1] - e[i]) /Ts # Calculation of Derivative of Error
        m_cont[i+1] = m_cont[i] + Ts*(Kp* diff_e[i+1] + Ki * e[i+1] + (Kd/Ts)* (diff_e[i+1]-diff_e[i])) # Calculation of Controller Output
    return y[1:]

n = math.floor(Tf / Ts)
sol_dynamics = sol_pid_velocity(2.5, 5, 0.1,2*np.ones(n+1), 2, 2,Ti,Tf,Fs)
dynamics = pid_velocity(2.5, 5, 0.1,2*np.ones(n+1), 2, 2,Ti,Tf,Fs)
assert np.allclose(sol_dynamics, dynamics)
### END HIDDEN TESTS

#### 8.2 Limited PID Control Output

In the above implementation, the PID controller is allowed to take any range of values. Practically, this is not possible, the controller output is limited to a minimum and maximum value. The controller value can not go below the ```control_min``` and ```control_max```

**The tasks to be performed:**<br>
Modify the above models to take this into account
- Position Form Implementation of PID Controller With Limited PID Output (0.25 Mark)
- Velocity Form Implementation of PID Controller With Limited PID Output (0.25 Mark)

```Prompt: The advantage and disadvantages of both implementations should become clearer after limiting the controller output. Which controller implementation is better?```

In [ ]:
### 0.25 Mark
def pid_pos_lim(Kp, Ki, Kd,MIN_PID, MAX_PID,reference, K, tau_p,Ti,Tf,Fs):
    """
    Inputs:
        Kp: Proportional gain
        Ki: Integral gain
        Kp: Derivative gain
        MIN_PID: Minimum value of PID controller output
        MAX_PID: Maximum value of PID controller output
        reference: Reference of the system (numpy array) Length: floor((Tf- Ti)*Fs)+1
        K: Gain of plant
        tau_p: Time constant of the plant
        Ti: Initial time
        Tf: Final Time
        Fs: Sampling frequency
    Outputs:
        Plant Response. (numpy array) Note that it must be in discrete domain
                                      Length: floor((Tf- Ti)*Fs)
    
    """
    ### BEGIN SOLUTION
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    m_cont = np.zeros(n+1)
    y = np.zeros(n+1)
    e = np.zeros(n+1)
    int_e = np.zeros(n+1)
    diff_e = np.zeros(n+1)
    
    for i in range(0,n):
        y[i+1] = (1 - (Ts/tau_p))*y[i] + (K*Ts/tau_p)*m_cont[i]
        e[i+1] = reference[i+1] - y[i+1] # Calculation of Error
        int_e[i+1] = int_e[i] + Ts* e[i+1] # Calculation of Integral of Error
        diff_e[i+1] = (e[i+1] - e[i]) /Ts # Calculation of Derivative of Error
        m_cont[i+1] = Kp* e[i+1] + Ki * int_e[i+1] + Kd* diff_e[i+1] # Calculation of Controller Output
        if(m_cont[i+1] > MAX_PID):
            m_cont[i+1] = MAX_PID
        elif(m_cont[i+1] < MIN_PID):
            m_cont[i+1] = MIN_PID
            
    y = y[1:]
    return y
    ### END SOLUTION

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
pid_pos_lim_dynamics= np.load('pid_pos_lim_dynamics.npy')
reference = np.load('reference.npy')
dynamics = pid_pos_lim(5, 5, 0.1,-50,50,reference, 1, 2,Ti,Tf,Fs)
assert np.allclose(pid_pos_lim_dynamics, dynamics)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS
def sol_pid_pos_lim(Kp, Ki, Kd,MIN_PID, MAX_PID,reference, K, tau_p,Ti,Tf,Fs):
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    m_cont = np.zeros(n+1)
    y = np.zeros(n+1)
    e = np.zeros(n+1)
    int_e = np.zeros(n+1)
    diff_e = np.zeros(n+1)
    
    for i in range(0,n):
        y[i+1] = (1 - (Ts/tau_p))*y[i] + (K*Ts/tau_p)*m_cont[i]
        e[i+1] = reference[i+1] - y[i+1] # Calculation of Error
        int_e[i+1] = int_e[i] + Ts* e[i+1] # Calculation of Integral of Error
        diff_e[i+1] = (e[i+1] - e[i]) /Ts # Calculation of Derivative of Error
        m_cont[i+1] = Kp* e[i+1] + Ki * int_e[i+1] + Kd* diff_e[i+1] # Calculation of Controller Output
        if(m_cont[i+1] > MAX_PID):
            m_cont[i+1] = MAX_PID
        elif(m_cont[i+1] < MIN_PID):
            m_cont[i+1] = MIN_PID
            
    return y[1:]

n = math.floor(Tf / Ts)
sol_dynamics = sol_pid_pos_lim(2.5, 5, 0.1,-25,25,2*np.ones(n+1), 2, 2,Ti,Tf,Fs)
dynamics = pid_pos_lim(2.5, 5, 0.1,-25,25,2*np.ones(n+1), 2, 2,Ti,Tf,Fs)

assert np.allclose(sol_dynamics, dynamics)
### END HIDDEN TESTS

In [ ]:
### 0.25 Mark
def pid_velocity_lim(Kp, Ki, Kd,MIN_PID,MAX_PID,reference, K, tau_p,Ti,Tf,Fs):
    """
    Inputs:
        Kp: Proportional gain
        Ki: Integral gain
        Kp: Derivative gain
        MIN_PID: Minimum value of PID controller output
        MAX_PID: Maximum value of PID controller output
        reference: Reference of the system (numpy array) Length: floor((Tf- Ti)*Fs)+1
        K: Gain of plant
        tau_p: Time constant of the plant
        Ti: Initial time
        Tf: Final Time
        Fs: Sampling frequency
    Outputs:
        Plant Dyanmics. (numpy array) Note that it must be in discrete domain
                                      Length: floor((Tf- Ti)*Fs)
    
    """
    ### BEGIN SOLUTION
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    m_cont = np.zeros(n+1)
    y = np.zeros(n+1)
    e = np.zeros(n+1)
    int_e = np.zeros(n+1)
    diff_e = np.zeros(n+1)
    
    for i in range(0,n):
        y[i+1] = (1 - (Ts/tau_p))*y[i] + (K*Ts/tau_p)*m_cont[i]
        e[i+1] = reference[i+1] - y[i+1] # Calculation of Error
        int_e[i+1] = int_e[i] + Ts* e[i+1] # Calculation of Integral of Error
        diff_e[i+1] = (e[i+1] - e[i]) /Ts # Calculation of Derivative of Error
        m_cont[i+1] = m_cont[i] + Ts*(Kp* diff_e[i+1] + Ki * e[i+1] + (Kd/Ts)* (diff_e[i+1]-diff_e[i])) # Calculation of Controller Output
        if(m_cont[i+1] > MAX_PID):
            m_cont[i+1] = MAX_PID
        elif(m_cont[i+1] < MIN_PID):
            m_cont[i+1] = MIN_PID
    y =  y[1:]
    return y
    ### END SOLUTION

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
pid_velocity_lim_dynamics= np.load('pid_velocity_lim_dynamics.npy')
reference = np.load('reference.npy')
dynamics = pid_velocity_lim(5, 5, 0.1,-50,50,reference, 1, 2,Ti,Tf,Fs)
assert np.allclose(pid_velocity_lim_dynamics, dynamics)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS
def sol_pid_velocity_lim(Kp, Ki, Kd,MIN_PID,MAX_PID,reference, K, tau_p,Ti,Tf,Fs):
    Ts = 1 / Fs
    n = math.floor((Tf-Ti)*Fs) 
    m_cont = np.zeros(n+1)
    y = np.zeros(n+1)
    e = np.zeros(n+1)
    int_e = np.zeros(n+1)
    diff_e = np.zeros(n+1)
    
    for i in range(0,n):
        y[i+1] = (1 - (Ts/tau_p))*y[i] + (K*Ts/tau_p)*m_cont[i]
        e[i+1] = reference[i+1] - y[i+1] # Calculation of Error
        int_e[i+1] = int_e[i] + Ts* e[i+1] # Calculation of Integral of Error
        diff_e[i+1] = (e[i+1] - e[i]) /Ts # Calculation of Derivative of Error
        m_cont[i+1] = m_cont[i] + Ts*(Kp* diff_e[i+1] + Ki * e[i+1] + (Kd/Ts)* (diff_e[i+1]-diff_e[i])) # Calculation of Controller Output
        if(m_cont[i+1] > MAX_PID):
            m_cont[i+1] = MAX_PID
        elif(m_cont[i+1] < MIN_PID):
            m_cont[i+1] = MIN_PID
    return y[1:]

n = math.floor((Tf-Ti) / Ts)
sol_dynamics = sol_pid_velocity_lim(2.5, 5, 0.1,-25,25,2*np.ones(n+1), 2, 2,Ti,Tf,Fs)
dynamics = pid_velocity_lim(2.5, 5, 0.1,-25,25,2*np.ones(n+1), 2, 2,Ti,Tf,Fs)

assert np.allclose(sol_dynamics, dynamics)
### END HIDDEN TESTS

### 9-Characteristics (1 Mark)
A control system is characterised by its step response. Using this, certain charactersistics are derived such as its settling time, rise time and peak overshoot.


<center><img src="characteristics.jpg" width="512" height="200"></center>

**The tasks to be performed:**<br>
Find the following characteristics
- Peak Overshoot (0.25 Marks)
- Rise Time (0.25 Mark)
- Settling Time (0.5 Marks)

Hint: Peak is positive only. Peak overshoot **percentage** is the maximum deviation from the value at inifinity (reference) in percentage of the final value at inifinity (reference

Hint: Settling time is the time in which the output is within a tolerance band (Look at figure)

Hint: Rise time is defined as the time taken to reach 90 % of the final value

Assumptions that can be made: System is stable

In [ ]:
### 0.25 Mark
def peak_overshoot(response,reference):
    """
    Inputs:
        response: Reponse of a system (numpy array) Length: floor((Tf- Ti)*Fs)
        reference: System Reference (numpy array) Length: floor((Tf- Ti)*Fs)+1
    Outputs:
       Percentage of Peak Overshoot (Normalized by response value at infinity)
    
    """
    ### BEGIN SOLUTION
    M = max(response)
    Mp = ((M-reference[-1])/reference[-1])*100
    return Mp
    ### END SOLUTION

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
reference = np.load('reference.npy')
sol_Mp = 6.628417857787472
Mp = peak_overshoot(pid_velocity(5, 5, 0.1,reference, 1, 2,Ti,Tf,Fs),reference)
assert np.allclose(sol_Mp, Mp)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS
def sol_peak_overshoot(response,reference):
    M = max(response)
    return ((M-reference[-1])/reference[-1])*100

sol_Mp = sol_peak_overshoot(sol_pid_velocity_lim(2.5, 5, 0.1,-25,25,2*np.ones(n+1), 2, 2,Ti,Tf,Fs),2*np.ones(n+1))
Mp = peak_overshoot(sol_pid_velocity_lim(2.5, 5, 0.1,-25,25,2*np.ones(n+1), 2, 2,Ti,Tf,Fs),2*np.ones(n+1))

assert np.allclose(sol_Mp,Mp)
### END HIDDEN TESTS

In [ ]:
### 0.5 Mark
def settling_time(response,reference,threshold,Ti,Tf,Fs):
    """
    Inputs:
        response: Reponse of a system (numpy array) Length: floor((Tf- Ti)*Fs)
        reference: System Reference (numpy array) Length: floor((Tf- Ti)*Fs)+1
        threshold: percentage oscillation (lee-way) allowed for settling
    Outputs:
       Settling time. Note it should be in analog domain (seconds)
    
    """
    ### BEGIN SOLUTION
    MIN_T = (1 - threshold*0.01) * reference[-1]
    MAX_T = (1 + threshold*0.01) * reference[-1]
    y = ((response>MIN_T)& (response<MAX_T))
    n = math.floor((Tf-Ti) *Fs)
    nt = np.where(y==1)
    
    for _,i in enumerate(nt[0]):
        if(y[i:].sum() == (n-i)):
            break
    settling_time =  (i/Fs) - Ti
    return settling_time
    ### END SOLUTION

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
reference = np.load('reference.npy')
sol_settling_t = 3.24
settling_t = settling_time(pid_velocity_lim(5, 5, 0.1,-50,50,reference, 1, 2,Ti,Tf,Fs),reference,20,Ti,Tf,Fs)
assert np.allclose(sol_settling_t, settling_t)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

def sol_settling_time(response,reference,threshold,Ti,Tf,Fs):
    MIN_T = (1 - threshold*0.01) * reference[-1]
    MAX_T = (1 + threshold*0.01) * reference[-1]
    y = ((response>MIN_T)& (response<MAX_T))
    n = math.floor((Tf-Ti) *Fs)
    nt = np.where(y==1)
    
    for _,i in enumerate(nt[0]):
        if(y[i:].sum() == (n-i)):
            break
    return (i/Fs) - Ti
   
sol_settling_t = sol_settling_time(sol_pid_velocity_lim(2.5, 5, 0.1,-25,25,2*np.ones(n+1), 2, 2,Ti,Tf,Fs),2*np.ones(n+1),20,Ti,Tf,Fs)
settling_t = settling_time(sol_pid_velocity_lim(2.5, 5, 0.1,-25,25,2*np.ones(n+1), 2, 2,Ti,Tf,Fs),2*np.ones(n+1),20,Ti,Tf,Fs)

assert np.allclose(sol_settling_t,settling_t)
### END HIDDEN TESTS

In [ ]:
### 0.25 Mark
def rise_time(response,reference,Ti,Tf,Fs):
    """
    Inputs:
        response: Reponse of a system (numpy array) Length: floor((Tf- Ti)*Fs)
        reference: System Reference (numpy array) Length: floor((Tf- Ti)*Fs)+1
    Outputs:
        Rise time. Note it should be in analog domain (seconds)
    
    """
    ### BEGIN SOLUTION
    y = (response> 0.9 * reference[-1])
    n = math.floor((Tf-Ti) *Fs)
    nt = np.where(y==1)
    rise_time =  nt[0][0]/Fs - Ti
    return rise_time
    ### END SOLUTION

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
reference = np.load('reference.npy')
sol_tr = 3.641
tr = rise_time(pid_velocity_lim(5, 5, 0.1,-50,50,reference, 1, 2,Ti,Tf,Fs),reference,Ti,Tf,Fs)
assert np.allclose(sol_tr, tr)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS
def sol_rise_time(response,reference,Ti,Tf,Fs):
    y = (response> 0.9 * reference[-1])
    n = math.floor((Tf-Ti) *Fs)
    nt = np.where(y==1)
    return nt[0][0]/Fs - Ti
   
sol_tr = sol_rise_time(sol_pid_velocity_lim(2.5, 5, 0.1,-25,25,2*np.ones(n+1), 2, 2,Ti,Tf,Fs),2*np.ones(n+1),Ti,Tf,Fs)
tr = rise_time(sol_pid_velocity_lim(2.5, 5, 0.1,-25,25,2*np.ones(n+1), 2, 2,Ti,Tf,Fs),2*np.ones(n+1),Ti,Tf,Fs)

assert np.allclose(sol_tr,tr)
### END HIDDEN TESTS

### 10- GA Tuned PID Remix (1 Mark)

Ah, you thought that was all, didn't you? Well, it's not over till it's over. Or something. Anyways, you may have noticed that although we've covered everything to run the genetic algorithm on our little all-ones problem or guessing a target number, there's still a long way to go before we can apply this technique in complicated scanerios like solving the travelling salesman problem or developing antennae for NASA.

To give you a glimpse of the power of genetic algorithms, consider the problem of tuning the parameters of a PID controller. If you don't want to do it yourself, and you don't want to spend time doing a grid search, you should consider using a genetic algorithm for this. It's basically tailor-made for tasks like these, where you have to optimize output based on multiple inputs. 

#### 10.1 GA Tuned PID Error Function (0.5 Mark)

Tune the $K_p, K_i, K_d$ values of a PID controller using a GA. To do that, the fitness function needs to be designed. The error should be the sum of the peak overshoot and integral time absolute error

$e = reference - \textit{output dynamics}$

$ITAE = \sum (t \cdot | e |)$

In [ ]:
### 0.5 Mark
def PID_tune_error_function(Kp, Ki, Kd,reference,K,tau_p,Ti,Tf,Fs):
    """
    Inputs:
        Kp: Proportional gain
        Ki: Integral gain
        Kp: Derivative gain
        reference: Reference of the system (numpy array) Length: floor((Tf- Ti)*Fs)+1
        K: Gain of plant
        tau_p: Time constant of the plant
        Ti: Initial time
        Tf: Final Time
        Fs: Sampling frequency
    Outputs:
        Error: As defined above
    
    """
    Ts = 1/Fs
    dynamics = pid_velocity(Kp, Ki, Kd, reference, K, tau_p, Ti, Tf, Fs)
    ### BEGIN SOLUTION
    e  = reference[-1] - dynamics # Broadcasting used # Error
    t = np.arange(Ti,Tf,Ts)
    
    Mp = peak_overshoot(dynamics,reference)
    
    J = sum(t*abs(e)) + Mp # Integral Time Absolute Error scaled by sampling period + Peak Overshoot -> Want to minimize both!
    return J
    ### END SOLUTION
    

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
reference = np.load('reference.npy')
sol_J = 280.06633899734993
J = PID_tune_error_function(5,5, 0.1,reference,2,2,Ti,Tf,Fs)
assert np.allclose(sol_J, J)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS
def sol_PID_tune_error_function(Kp, Ki, Kd,reference,K,tau_p,Ti,Tf,Fs):
    Ts = 1/Fs
    dynamics = pid_velocity(Kp, Ki, Kd,reference, K, tau_p,Ti,Tf,Fs)
    e  = reference[-1] - dynamics # Broadcasting used # Error
    t = np.arange(Ti,Tf,Ts)
    Mp = peak_overshoot(dynamics,reference)
    J = sum(t*abs(e)) + Mp # Integral Time Absolute Error scaled by sampling period + Peak Overshoot -> Want to minimize both!
    return J
    
   
sol_J = sol_PID_tune_error_function(5,5, 0.1,3*np.ones(math.floor((Tf-Ti)*Fs)+1),2,2,Ti,Tf,Fs)
J = PID_tune_error_function(5,5, 0.1,3*np.ones(math.floor((Tf-Ti)*Fs)+1),2,2,Ti,Tf,Fs)

assert np.allclose(sol_J,J)
### END HIDDEN TESTS

#### 10.2 GA Fitness Function (0.5 Mark)

For creating the correct fitness function, we need to consider the problem statement carefully. The values we need are going to be floating point, positive, and relatively small. Luckily for us, we already have a function which can convert the bitstring of our _GAString_ to a floating point number in a particular range.

Assume the following - 
- $0.0 \leq K _p \leq 20.0$
- $0.0 \leq K _i \leq 10.0$
- $0.0 \leq K _d \leq 1.0$
- The number of bits required for each individual parameter is 10.

For the actual fitness, just use the arithmetic inverse of the value obtained from the PID error function.

**NOTE**: Do not use extra bits in the GAString. Keep the bits for each value separate and in the same order i.e. $K_p, K_i, K_d$

In [ ]:
### 0.5 Mark
def usefulFitness(dude):
    """
    Input:  dude is a GAString object i.e. an individual chromosome
    
    Output: the fitness, as specified above in section 10.1
    """
    ### BEGIN SOLUTION
    Kp = decodeFloatRange(dude.genes[:10],0.0,20.0)
    Ki = decodeFloatRange(dude.genes[10:20],0.0,10.0)
    Kd = decodeFloatRange(dude.genes[20:],0.0,1.0)
    
    #PID charateristic to be optimized
    pid_output = PID_tune_error_function(Kp, Ki, Kd, reference, K, tau_p, Ti, Tf, Fs)
    
    #If any transforms need to be made for converting to fitness
    fitness = -pid_output
    
    return fitness
    ### END SOLUTION

In [ ]:
# do not change code here
### SAMPLE TEST CASE
print("Running sample test case")
random.seed(1002)
reference = np.load('reference.npy')
K = 2
tau_p = 2
res = usefulFitness(GAString(30))
ans = -110.4321552444553
assert np.isclose(ans,res)
print("Sample test case passed")

In [ ]:
# do not change code here
# Hidden Test cases
### BEGIN HIDDEN TESTS

random.seed()

def _funkyfoo(dude):
    Kp = decodeFloatRange(dude.genes[:10],0.0,20.0)
    Ki = decodeFloatRange(dude.genes[10:20],0.0,10.0)
    Kd = decodeFloatRange(dude.genes[20:],0.0,1.0)
    return -PID_tune_error_function(Kp, Ki, Kd, reference, K, tau_p, Ti, Tf, Fs)

for _ in range(5):
    dude = GAString(30)
    
    res = usefulFitness(dude)
    ans = _funkyfoo(dude)
    assert np.isclose(ans,res)

### END HIDDEN TESTS

In [ ]:
# playground
# fill in the blanks before running
random.seed(9000)
print("Running actual use case")

reference = np.load('reference.npy')
K = 2
tau_p = 2

def show_graph(Kp, Ki, Kd):
    dynamics = pid_velocity(Kp, Ki, Kd, reference, K, tau_p, Ti, Tf, Fs)
    t = np.arange(Ti,Tf,Ts)
    fig = plt.figure()
    ax = plt.subplot(111)
    ax.plot(t,dynamics,'b-',
            t,reference[:-1],'r--')
    plt.show()

pop = initPop(40,__)
f = usefulFitness
s0 = 40
s1 = 2
s2 = 32
s = selectionTournament
c = crossoverTwoPoint
m = mutationInversion

for i in range(20):
    fit, dude = getBestDude(pop,f)
    
    ### blanks
    Kp = __
    Ki = __
    Kd = __
    
    show_graph(Kp,Ki,Kd)
    print(f"Kp = {Kp:.5f} | Ki = {Ki:.5f} | Kd = {Kd:.5f}")
    print(f"Chromosome = {dude}\nFitness = {fit}")
    
    pop = singleIteration(pop,f,s0,s1,s2,s,c,m)

fit, dude = getBestDude(pop,f)
show_graph(Kp,Ki,Kd)
print(f"Kp = {Kp:.5f} | Ki = {Ki:.5f} | Kd = {Kd:.5f}")
print(f"Chromosome = {dude}\nFitness = {fit}")

Fin.